In [5]:
from dotenv import load_dotenv
import logging
import os
from datetime import datetime

from adobe.pdfservices.operation.auth.service_principal_credentials import ServicePrincipalCredentials
from adobe.pdfservices.operation.exception.exceptions import ServiceApiException, ServiceUsageException, SdkException
from adobe.pdfservices.operation.io.stream_asset import StreamAsset
from adobe.pdfservices.operation.pdf_services import PDFServices
from adobe.pdfservices.operation.pdf_services_media_type import PDFServicesMediaType
from adobe.pdfservices.operation.pdfjobs.jobs.split_pdf_job import SplitPDFJob
from adobe.pdfservices.operation.pdfjobs.params.split_pdf.split_pdf_params import SplitPDFParams
from adobe.pdfservices.operation.pdfjobs.result.split_pdf_result import SplitPDFResult

# Initialize the logger
logging.basicConfig(level=logging.INFO)
load_dotenv()

True

In [6]:
file = open('src/resources/splitPDFInput.pdf', 'rb')
input_stream = file.read()
file.close()

# Initial setup, create credentials instance
credentials = ServicePrincipalCredentials(
    client_id=os.getenv('PDF_SERVICES_CLIENT_ID'),
    client_secret=os.getenv('PDF_SERVICES_CLIENT_SECRET')
)
# Creates a PDF Services instance
pdf_services = PDFServices(credentials=credentials)

In [8]:
# Creates a PDF Services instance
pdf_services = PDFServices(credentials=credentials)

# Creates an asset(s) from source file(s) and upload
input_asset = pdf_services.upload(input_stream=input_stream,
                                    mime_type=PDFServicesMediaType.PDF)

# Create parameters for the job
split_pdf_params = SplitPDFParams(page_count=1)

# Creates a new job instance
split_pdf_job = SplitPDFJob(input_asset, split_pdf_params)

# Submit the job and gets the job result
location = pdf_services.submit(split_pdf_job)
pdf_services_response = pdf_services.get_job_result(location, SplitPDFResult)

# Get content from the resulting asset(s)
result_assets = pdf_services_response.get_result().get_assets()

INFO:adobe.pdfservices.operation.internal.pdf_services_helper:Started uploading asset
INFO:adobe.pdfservices.operation.internal.pdf_services_helper:Finished uploading asset
INFO:adobe.pdfservices.operation.internal.pdf_services_helper:Started submitting SPLIT_PDF job
INFO:adobe.pdfservices.operation.internal.pdf_services_helper:Started getting job result
INFO:adobe.pdfservices.operation.internal.pdf_services_helper:Finished polling for status
INFO:adobe.pdfservices.operation.internal.pdf_services_helper:Finished getting job result


In [10]:
# Creates an output stream and copy stream asset's content to it
now = datetime.now()
time_stamp = now.strftime("%Y-%m-%dT%H-%M-%S")
os.makedirs("output/SplitPDFByNumberOfPages", exist_ok=True)
output_file_path = "output/SplitPDFByNumberOfPages/split{time_stamp}.pdf"

for i, result_asset in enumerate(result_assets):
    stream_asset: StreamAsset = pdf_services.get_content(result_asset)
    with open(f"{output_file_path}_{i}.pdf", "wb") as file:
        file.write(stream_asset.get_input_stream())

INFO:adobe.pdfservices.operation.internal.pdf_services_helper:Started getting content
INFO:adobe.pdfservices.operation.internal.pdf_services_helper:Finished getting content
INFO:adobe.pdfservices.operation.internal.pdf_services_helper:Started getting content
INFO:adobe.pdfservices.operation.internal.pdf_services_helper:Finished getting content
INFO:adobe.pdfservices.operation.internal.pdf_services_helper:Started getting content
INFO:adobe.pdfservices.operation.internal.pdf_services_helper:Finished getting content
INFO:adobe.pdfservices.operation.internal.pdf_services_helper:Started getting content
INFO:adobe.pdfservices.operation.internal.pdf_services_helper:Finished getting content
